## Molkit

In [2]:
# import libraries
import numpy as np
from moha import HamHeisenberg

# 3-site exchange and axial couplings
J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]
                 ])

mu = np.zeros(2)

# Define the Hamiltonian for 3 sites
ham = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham.generate_one_body_integral(dense=True)
two_body = ham.generate_two_body_integral(dense=True)

In [10]:
import sys
import os
import sys  
sys.path.insert(0, '../')
from moha.molkit.hamiltonians import MolHam
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)

spin_one, spin_two = Molecular_Hamiltonian.spinize_H()

In [16]:
# ------------------------------------------------------------------
# 1) Set up the 3-site Heisenberg model exactly as you did
# ------------------------------------------------------------------
import numpy as np
from moha import HamHeisenberg
from moha.molkit.hamiltonians import MolHam  
from moha.molkit.utils.tools import from_geminal

J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]])
mu = np.zeros(2)

ham_hei  = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham_hei.generate_one_body_integral(dense=True)
two_body = ham_hei.generate_two_body_integral(dense=True)
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)

h1_spin, h2_spin = Molecular_Hamiltonian.spinize_H()



G = MolHam.to_geminal(h2_spin, type='h2')            # shape: (n_gem, n_gem)